# 컴피티션 개인전
- 17시30분 마감
- 평가지표: AUC
- 1~3등에게 스타벅스 쿠폰을 전달

# 데이터 다운로드

In [1913]:
!gdown 1BBRlH5Yxgh4Y2-BRJNvoXhswEoI918Nr
!unzip -oqq campaign.zip

Downloading...
From: https://drive.google.com/uc?id=1BBRlH5Yxgh4Y2-BRJNvoXhswEoI918Nr
To: /content/campaign.zip
100% 60.2k/60.2k [00:00<00:00, 55.9MB/s]


# 데이터 경로와 시드값

In [1914]:
DATA_PATH = "./"
SEED = 42

# 데이터 불러오기

In [1915]:
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}campaign_train.csv")
test = pd.read_csv(f"{DATA_PATH}campaign_test.csv")
train.shape ,test.shape

((1344, 27), (896, 26))

In [1916]:
display(train.head())
display(train.info())

,ID,출생연도,고객_교육수준,고객_결혼여부,고객_소득,고객_자녀수,고객_청소년수,고객_가입날짜,고객_최신구매일_경과기간,고객_와인_구매금액,...,고객_카탈로그_통한_구매횟수,고객_매장방문_구매횟수,고객_지난달_회사사이트_방문횟수,캠페인1_수락여부,캠페인2_수락여부,캠페인3_수락여부,캠페인4_수락여부,캠페인5_수락여부,불만제기,target
0,train_0,1969,박사,기혼,30396.0,1,0,2014-04-30,22,15,...,1,2,7,0,0,1,0,0,0,1
1,train_1,1967,박사,기혼,36947.0,1,1,2012-08-07,49,88,...,0,4,9,0,0,0,0,0,0,0
2,train_2,1949,석사,사별,47570.0,1,1,2013-05-29,3,67,...,2,2,7,0,0,0,0,0,0,1
3,train_3,1976,석사,기혼,81929.0,1,0,2012-09-29,60,1486,...,4,10,6,0,0,1,0,1,0,1
4,train_4,1982,고졸,기혼,57937.0,0,1,2014-02-16,56,261,...,4,9,3,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 1344 non-null   object 
 1   출생연도               1344 non-null   int64  
 2   고객_교육수준            1331 non-null   object 
 3   고객_결혼여부            1337 non-null   object 
 4   고객_소득              1326 non-null   float64
 5   고객_자녀수             1344 non-null   int64  
 6   고객_청소년수            1344 non-null   int64  
 7   고객_가입날짜            1344 non-null   object 
 8   고객_최신구매일_경과기간      1344 non-null   int64  
 9   고객_와인_구매금액         1344 non-null   int64  
 10  고객_과일_구매금액         1344 non-null   int64  
 11  고객_육류_구매금액         1344 non-null   int64  
 12  고객_생선_구매금액         1344 non-null   int64  
 13  고객_사탕_구매금액         1344 non-null   int64  
 14  고객_골드_구매금액         1344 non-null   int64  
 15  고객_할인품목_구매횟수       1344 non-null   int64  
 16  고객_회사사이트_통한_구매횟수   1344 

None

# 특성 공학
- 수행해주세요!!

In [1917]:
train_ft = train.iloc[:,1:-1].copy()
test_ft = test.iloc[:,1:].copy()

train_ft.shape, test_ft.shape

((1344, 25), (896, 25))

In [1918]:
from datetime import datetime

train_ft['고객_나이'] = datetime.now().year - train_ft['출생연도']
test_ft['고객_나이'] = datetime.now().year - test_ft['출생연도']
train_ft = train_ft.drop(columns="출생연도")
test_ft = test_ft.drop(columns="출생연도")

In [1919]:
cols = train_ft.columns[train_ft.columns.str.contains("수락여부")]
train_ft['총_캠페인_수락횟수'] = train_ft[cols].sum(axis=1)
test_ft['총_캠페인_수락횟수'] = test_ft[cols].sum(axis=1)

In [1920]:
cols = train_ft.columns[train_ft.columns.str.contains("구매금액")]
train_ft['총_구매금액'] = train_ft[cols].sum(axis=1)
test_ft['총_구매금액'] = test_ft[cols].sum(axis=1)

In [1921]:
train_ft['고객_가입날짜'] = pd.to_datetime(train_ft['고객_가입날짜'])
train_ft['가입일수'] = (datetime.now() - train_ft['고객_가입날짜']).dt.days
test_ft['고객_가입날짜'] = pd.to_datetime(test_ft['고객_가입날짜'])
test_ft['가입일수'] = (datetime.now() - test_ft['고객_가입날짜']).dt.days

In [1922]:
cols = ["고객_회사사이트_통한_구매횟수", "고객_카탈로그_통한_구매횟수", "고객_매장방문_구매횟수"]
train_ft["총_구매횟수"] = train_ft[cols].sum(axis=1)
test_ft["총_구매횟수"] = test_ft[cols].sum(axis=1)

In [1923]:
# agg_dict = {
#     "고객_소득": [
#         ("교육_결혼별_소득_평균", "mean"),
#     ]
# }
# tmp = train_ft.groupby(["고객_교육수준", "고객_결혼여부"]).agg(agg_dict).reset_index()
# tmp.columns = ["고객_교육수준", "고객_결혼여부", "교육_결혼별_소득_평균"]
# train_ft = train_ft.merge(tmp, on=["고객_교육수준", "고객_결혼여부"], how="left")
# test_ft = test_ft.merge(tmp, on=["고객_교육수준", "고객_결혼여부"], how="left")

In [1924]:
# train_ft["자녀_청소년_비율"] = train_ft["고객_청소년수"] / train_ft["고객_자녀수"] + 0.00001
# test_ft["자녀_청소년_비율"] = test_ft["고객_청소년수"] / test_ft["고객_자녀수"] + 0.00001

In [1925]:
train_ft["1회_구매금액"] = np.where(train_ft["총_구매횟수"] == 0, 0, train_ft["총_구매금액"] / train_ft["총_구매횟수"])
test_ft["1회_구매금액"] = np.where(test_ft["총_구매횟수"] == 0, 0, test_ft["총_구매금액"] / test_ft["총_구매횟수"])

In [1926]:
train_ft["고객_가입연도"] = train_ft["고객_가입날짜"].dt.year
train_ft["고객_가입월"] = train_ft["고객_가입날짜"].dt.month

test_ft["고객_가입연도"] = test_ft["고객_가입날짜"].dt.year
test_ft["고객_가입월"] = test_ft["고객_가입날짜"].dt.month

# 결측치 처리

In [1927]:
display(train_ft.isnull().sum())
display(test_ft.isnull().sum())

,0
고객_교육수준,13
고객_결혼여부,7
고객_소득,18
고객_자녀수,0
고객_청소년수,0
고객_가입날짜,0
고객_최신구매일_경과기간,0
고객_와인_구매금액,0
고객_과일_구매금액,0
고객_육류_구매금액,0


,0
고객_교육수준,5
고객_결혼여부,0
고객_소득,6
고객_자녀수,0
고객_청소년수,0
고객_가입날짜,0
고객_최신구매일_경과기간,0
고객_와인_구매금액,0
고객_과일_구매금액,0
고객_육류_구매금액,0


In [1928]:
e_level = train_ft["고객_교육수준"].mode()[0]
m_status = train_ft["고객_결혼여부"].mode()[0]
s_mean = train_ft["고객_소득"].mean()

e_level, m_status, s_mean

('학사', '기혼', 52128.450980392154)

In [1929]:
train_ft["고객_교육수준"] = train_ft["고객_교육수준"].fillna(e_level)
train_ft["고객_결혼여부"] = train_ft["고객_결혼여부"].fillna(m_status)
train_ft["고객_소득"] = train_ft["고객_소득"].fillna(s_mean)

test_ft["고객_교육수준"] = test_ft["고객_교육수준"].fillna(e_level)
test_ft["고객_결혼여부"] = test_ft["고객_결혼여부"].fillna(m_status)
test_ft["고객_소득"] = test_ft["고객_소득"].fillna(s_mean)

train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [1930]:
train_ft = train_ft.drop(columns="고객_가입날짜")
test_ft = test_ft.drop(columns="고객_가입날짜")

# 피처 인코딩

In [1931]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [1932]:
# map_dict = {
#     "초졸_중졸": 1,
#     "고졸": 2,
#     "학사": 3,
#     "석사": 4,
#     "박사": 5
# }

# train_ft["고객_교육수준"] = train_ft["고객_교육수준"].map(map_dict)
# test_ft["고객_교육수준"] = test_ft["고객_교육수준"].map(map_dict)

In [1933]:
cols = train_ft.select_dtypes("object").columns
cols

Index(['고객_교육수준', '고객_결혼여부'], dtype='object')

In [1934]:
enc = LabelEncoder()
for col in cols:
    train_ft[col] = enc.fit_transform(train_ft[col])
    test_ft[col] = enc.transform(test_ft[col])

train_ft.shape, test_ft.shape

((1344, 31), (896, 31))

# 피처 스케일링

In [1935]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer

In [1936]:
scaler = MinMaxScaler()
scaler.fit(train_ft)

MinMaxScaler()

In [1937]:
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [1938]:
target = train["target"]
target.mean()

0.1488095238095238

# cv 점수 확인하기

In [1939]:
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

cv = KFold(5, random_state=SEED, shuffle=True)

class Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        # hp = {
        #     "random_state": self.seed,
        #     "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        #     "max_depth": trial.suggest_int("max_depth", 3, 15),
        #     "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
        #     "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        #     "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
        #     "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        #     "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0)
        # }
        # model = LGBMClassifier(**hp)
        hp = {
            "random_state": self.seed,
            "n_estimators": trial.suggest_int("n_estimators", 100, 300, step=10),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "gamma": trial.suggest_float("gamma", 0, 5)
        }
        model = XGBClassifier(**hp)
        score = cross_val_score(model, self.x, self.y, cv=self.cv, scoring="roc_auc", n_jobs=-1).mean()

        return score

In [1940]:
# %pip install optuna

In [1941]:
import optuna

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

objective_func = Objective(train_ft, target, SEED)

study.optimize(objective_func, n_trials=50)

[I 2024-10-28 09:06:19,343] A new study created in memory with name: no-name-352e4c54-eed2-4505-92d6-813b42fcb041
[I 2024-10-28 09:06:23,711] Trial 0 finished with value: 0.8835064572323965 and parameters: {'n_estimators': 170, 'max_depth': 10, 'learning_rate': 0.36626497696389115, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_weight': 2, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.8835064572323965.
[I 2024-10-28 09:06:24,605] Trial 1 finished with value: 0.8582258978214572 and parameters: {'n_estimators': 280, 'max_depth': 7, 'learning_rate': 0.3543282163202267, 'subsample': 0.608233797718321, 'colsample_bytree': 0.9879639408647978, 'min_child_weight': 9, 'gamma': 1.0616955533913808}. Best is trial 0 with value: 0.8835064572323965.
[I 2024-10-28 09:06:25,124] Trial 2 finished with value: 0.8964091029164282 and parameters: {'n_estimators': 130, 'max_depth': 4, 'learning_rate': 0.1528168792368093, 'subsample': 0.8099025726528951, 'co

In [1942]:
study.best_value, study.best_params

(0.9035765910600138,
 {'n_estimators': 190,
  'max_depth': 8,
  'learning_rate': 0.10119177284923926,
  'subsample': 0.8176910617283651,
  'colsample_bytree': 0.8323506941529332,
  'min_child_weight': 5,
  'gamma': 2.0048311882612464})

In [1943]:
model = XGBClassifier(**study.best_params)
# model = LGBMClassifier(**study.best_params)

# 모델 학습및 테스트 데이터 예측

In [1944]:
model.fit(train_ft, target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8323506941529332, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=2.0048311882612464,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.10119177284923926,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=190, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [1945]:
pred = model.predict_proba(test_ft)[:,1]
pred.shape

(896,)

# 예측 파일 저장후 컴피티션 페이지에 제출

In [1946]:
pd.DataFrame(pred, columns=["target"]).to_csv("권지혁.csv")